# Data Challenge
By: Eddy Trang

***
***

#### Tasks:
1. Search for Tencent’s official website from a search engine.
2. From the search engine results, find link to official website and use it to enter Tencent website to search for 2019 Tencent’s annual report
3. Download Tencent’s 2019 annual report, in pdf format, to the “Download” folder of your local computer.
4. Extract the information under the section “SUBSIDIARIES AND CONTROLLED STRUCTURED ENTITIES” but only the following columns are relevant:
   - Name of the principal subsidiaries
   - Particulars of issued/paid-in capital for each subsidiary
   - Proportion of equity interest held by the Group (%) for each subsidiary
5. Perform fuzzy matching on each subsidiary name against each of the other subsidiaries’ names. Fuzzy matching is a technique that matches strings together that fits a particular pattern approximately. You will need to record:
   - “Yes” if the fuzzy match is an “approximate match”, along with what the top matched name was to (i.e. the matched subsidiary name)
   - Note that you will need to define “approximate match” and explain your reasoning. Your answer can be placed in the output xlsx file in a separate tab.
   - “No” if the fuzzy match is not an approximate match
6. Output the results into the below five columns within a xlsx file:
   - Name
   - Capital
   - Interest
   - Match (Yes / No)
   - Matched subsidiary name
   
***
***

# Outline
#### I.    Format Working Space
#### II.   Necessary Imports
#### III.  Search for Tencent's 2019 Annual Report and Download
#### IV.  Data Extraction and Cleaning
#### V.   Fuzzy Matching and Export to Excel

***
***

## I. Format Working Space

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## II. Necessary Imports

In [ ]:
from selenium import webdriver
import time
import camelot
import numpy as np
import pandas as pd

## III. Search for Tencent's 2019 Annual Report and Download

**1. Set up ChromeDriver**

In [ ]:
chromedriver = "/Users/eddytrang/Downloads/chromedriver" # path to the chromedriver executable
chromeOptions = webdriver.ChromeOptions()
prefs = {"plugins.always_open_pdf_externally": True}
chromeOptions.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(chromedriver, options=chromeOptions)

**2. Set up search query**

In [ ]:
query = "tencent official website"
google_search = f"https://www.google.com.sg/search?q="
google_query = google_search + query.replace(' ', '+')
google_query

'https://www.google.com.sg/search?q=tencent+official+website'

**3. Search for report**

In [ ]:
#1. Perform Google search on "tencent official website"
driver.get(google_query)
time.sleep(5)  

#2. Click first link available
result = driver.find_element_by_tag_name('h3') #first result under h3
result.click()
time.sleep(5)

#3. Click "Accept all analytics cookies"
result2 = driver.find_element_by_xpath('/html/body/div/div[4]/div[3]/div[3]/div[1]')
result2.click()
time.sleep(2)

#4. Click "Investors"
result3 = driver.find_element_by_xpath('/html/body/div/div[1]/div[1]/ul/li[5]/a')
result3.click()
time.sleep(2)

# Click "2020 Annual Financial Report" to download
result4 = driver.find_element_by_xpath('//*[@id="investors-con-2"]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/a/div[1]/p')
result4.click()
time.sleep(2)

## IV. Data Extraction and Cleaning

### Table 1: Page 302

**1. Read in Table 1 and Convert to Dataframe**

In [ ]:
table1=camelot.read_pdf(r'/Users/eddytrang/Downloads/960eae1f18dd716fd3a7d704e123d7a5.pdf', flavor='stream',pages='302')
df1=table1[0].df

In [ ]:
df1

,0,1,2,3,4
0,Name,nature of legal entity,capital,by the Group (%),Principal activities and place of operation
1,Tencent Computer,"Established in the PRC,","RMB65,000,000",100%,Provision of value-added services and
2,,limited liability company,,(Note (a)),Internet advertisement services in the PRC
3,Tencent Technology,"Established in the PRC,","USD2,000,000",100%,Development of softwares and provision of
4,,wholly foreign owned,,,information technology services in the PRC
5,,enterprise,,,
6,Shenzhen Shiji Kaixuan Technology,"Established in the PRC,","RMB11,000,000",100%,Provision of Internet advertisement services
7,Company Limited,limited liability company,,(Note (a)),in the PRC
8,Tencent Cyber (Tianjin) Company Limited,"Established in the PRC,","USD90,000,000",100%,Development of softwares and provision of
9,,wholly foreign owned,,,information technology services in the PRC


**2. Clean Table 1**

In [ ]:
#1. Drop non-relevant columns: Placement of establishment and Principal Activities
df1 = df1.drop(df1.columns[[1, 4]], axis=1)

#2. Drop rows with no values and first row
df1[0].replace('', np.nan, inplace=True)
df1.dropna(subset=[0], inplace=True)
df1=df1.iloc[1:]

#3. Reset index and add in column names
df1 = df1.reset_index(drop=True)
df1.columns = ['Name','Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']

#4. Loop through df to move "Company Limited" where it belongs. When the table was read in, there were some cases where Camelot did not include "Company Limited" as part of the company name
for index in df1.index:
    if df1.loc[index,'Name'] == 'Company Limited':
        df1.loc[index-1,'Name_official'] = df1.loc[index-1,'Name'] + ' ' + df1.loc[index,'Name']
    else:
        df1.loc[index,'Name_official'] = df1.loc[index,'Name']
        
#5. Add column names and drop old "Name" column
df1.dropna(subset=['Name_official'], inplace=True)
df1.drop('Name', axis=1, inplace=True)
df1 = df1[['Name_official', 'Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']]

In [ ]:
df1

,Name_official,Particulars of issued/paid-in capital,Proportion of equity interest held by the Group(%)
0,Tencent Computer,"RMB65,000,000",100%
1,Tencent Technology,"USD2,000,000",100%
2,Shenzhen Shiji Kaixuan Technology Company Limited,"RMB11,000,000",100%
4,Tencent Cyber (Tianjin) Company Limited,"USD90,000,000",100%
5,Tencent Asset Management Limited,USD100,100%
6,Tencent Technology (Beijing) Company Limited,"USD1,000,000",100%
8,Nanjing Wang Dian Technology Company Limited,"RMB10,290,000",100%
10,Beijing BIZCOM Technology Company Limited,"RMB1,216,500,000",100%
12,Beijing Starsinhand Technology Company Limited,"RMB10,000,000",100%


### Table 2: Page 303


**1. Read in Table 2 and Convert to Dataframe**

In [ ]:
table2=camelot.read_pdf(r'/Users/eddytrang/Downloads/960eae1f18dd716fd3a7d704e123d7a5.pdf', flavor='stream',pages='303')
df2=table2[0].df
df2

,0,1,2,3,4
0,Name,nature of legal entity,capital,by the Group (%),Principal activities and place of operation
1,Tencent Cyber (Shenzhen),"Established in the PRC,","USD30,000,000",100%,Development of softwares in the PRC
2,Company Limited,wholly foreign owned,,,
3,,enterprise,,,
4,Tencent Technology (Shanghai),"Established in the PRC,","USD5,000,000",100%,Development of softwares and provision of
5,Company Limited,wholly foreign owned,,,information technology services in the PRC
6,,enterprise,,,
7,Tencent Technology (Chengdu),"Established in the PRC,","USD220,000,000",100%,Development of softwares and provision of
8,Company Limited,wholly foreign owned,,,information technology services in the PRC
9,,enterprise,,,


**2. Clean Table 2**

In [ ]:
#1. Drop non-relevant columns: Placement of establishment and Principal Activities
df2 = df2.drop(df2.columns[[1, 4]], axis=1)

#2. Drop rows with no values and first row
df2[0].replace('', np.nan, inplace=True)
df2.dropna(subset=[0], inplace=True)
df2=df2.iloc[1:]

#3. Reset index and add in column names
df2 = df2.reset_index(drop=True)
df2.columns = ['Name','Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']

#4. Loop through df to move "Company Limited" where it belongs. When the table was read in, there were some cases where Camelot did not include "Company Limited" as part of the company name
for index in df1.index:
    for index in df2.index:
        if df2.loc[index,'Name'] == 'Company Limited':
            df2.loc[index-1,'Name_official'] = df2.loc[index-1,'Name'] + ' ' + df2.loc[index,'Name']
        else:
            df2.loc[index,'Name_official'] = df2.loc[index,'Name']
        
#5. Add column names and drop old "Name" column
df2.dropna(subset=['Name_official'], inplace=True)
df2.drop('Name', axis=1, inplace=True)
df2 = df2[['Name_official', 'Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']]

In [ ]:
df2

,Name_official,Particulars of issued/paid-in capital,Proportion of equity interest held by the Group(%)
0,Tencent Cyber (Shenzhen) Company Limited,"USD30,000,000",100%
2,Tencent Technology (Shanghai) Company Limited,"USD5,000,000",100%
4,Tencent Technology (Chengdu) Company Limited,"USD220,000,000",100%
6,Tencent Technology (Wuhan) Company Limited,"USD30,000,000",100%
8,Tencent Cloud Computing (Beijing) Company Limited,"RMB1,042,500,000",100%
10,Morespark Limited,"HKD1,000",100%
11,Beijing Tencent Culture Media Company Limited,"RMB5,000,000",100%
13,"Riot Games, Inc.","USD1,306",100%
14,China Literature Limited,"USD101,578",57.57%*


### Table 3: Page 304

**1. Read in Table 3 and Convert to Dataframe**

In [ ]:
table3=camelot.read_pdf(r'/Users/eddytrang/Downloads/960eae1f18dd716fd3a7d704e123d7a5.pdf', flavor='stream',pages='304',table_areas=['0,600,1000,400'])
df3=table3[0].df
df3

,0,1,2,3,4,5
0,TME,,Established in the,"USD277,999",49.06%*,Provision of online music entertainment
1,,,"Cayman Islands,",,,services in the PRC
2,,,limited liability company,,,
3,Supercell Oy,,"Established in Finland,","EUR2,500",70.03%,Development and operation of mobile games
4,,,limited liability company,,,in Finland
5,Shenzhen Tencent Culture Media,,"Established in the PRC,","RMB5,000,000",100%,Design and production of advertisement
6,,Company Limited,limited liability company,,,in the PRC
7,*,on an outstanding basis,,,,


**2. Clean Table 2**

In [ ]:
#1. Move all strings in column 1 to column 0
df3[0] = df3[0] + df3[1]

#2. Drop non-relevant columns
df3 = df3.drop(df3.columns[[1, 2, 5]], axis=1)

#3. Drop rows with no values and first row
df3[0].replace('', np.nan, inplace=True)
df3.dropna(subset=[0], inplace=True)

#4. Reset index and add in column names
df3 = df3.reset_index(drop=True)
df3.columns = ['Name','Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']

#5. Loop through df to move "Company Limited" where it belongs. When the table was read in, there were some cases where Camelot did not include "Company Limited" as part of the company name
for index in df3.index:
    if df3.loc[index,'Name'] == 'Company Limited':
        df3.loc[index-1,'Name_official'] = df3.loc[index-1,'Name'] + ' ' + df3.loc[index,'Name']
    else:
        df3.loc[index,'Name_official'] = df3.loc[index,'Name']
        
#6. Add column names and drop old "Name" column
df3.dropna(subset=['Name_official'], inplace=True)
df3.drop('Name', axis=1, inplace=True)
df3 = df3[['Name_official', 'Particulars of issued/paid-in capital','Proportion of equity interest held by the Group(%)']]
df3=df3.iloc[:-1,:]


In [ ]:
df3

,Name_official,Particulars of issued/paid-in capital,Proportion of equity interest held by the Group(%)
0,TME,"USD277,999",49.06%*
1,Supercell Oy,"EUR2,500",70.03%
2,Shenzhen Tencent Culture Media Company Limited,"RMB5,000,000",100%


### Combine the 3 dataframes

In [ ]:
frames = [df1, df2, df3]
main_df = pd.concat(frames)
main_df = main_df.reset_index(drop=True)
main_df.rename(columns={'Name_official': 'Name'}, inplace=True)

In [ ]:
main_df

,Name,Particulars of issued/paid-in capital,Proportion of equity interest held by the Group(%)
0,Tencent Computer,"RMB65,000,000",100%
1,Tencent Technology,"USD2,000,000",100%
2,Shenzhen Shiji Kaixuan Technology Company Limited,"RMB11,000,000",100%
3,Tencent Cyber (Tianjin) Company Limited,"USD90,000,000",100%
4,Tencent Asset Management Limited,USD100,100%
5,Tencent Technology (Beijing) Company Limited,"USD1,000,000",100%
6,Nanjing Wang Dian Technology Company Limited,"RMB10,290,000",100%
7,Beijing BIZCOM Technology Company Limited,"RMB1,216,500,000",100%
8,Beijing Starsinhand Technology Company Limited,"RMB10,000,000",100%
9,Tencent Cyber (Shenzhen) Company Limited,"USD30,000,000",100%


## V. Fuzzy Matching and Export to Excel

**1. Necessary Imports**

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/eddytrang/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


**2. Add all company names to a list**

In [ ]:
company_list = main_df['Name'].to_list()

**3. Loop throgh "Name" column in main_df and perform fuzzy matching on each name**

In [ ]:
Match = []
Matched_subsidiary_name = []

for company_name in main_df['Name']:
    new_company_list = company_list.copy()
    new_company_list.remove(company_name)
    
    score = process.extractOne(company_name,new_company_list,scorer=fuzz.partial_ratio)
    
    if score[1] > 83:
        Match.append('Yes')
        Matched_subsidiary_name.append(score[0])
    else:
        Match.append('No')
        Matched_subsidiary_name.append('')
    
    new_company_list=[]
    
main_df['Match'] = Match
main_df['Matched subsidiary name'] = Matched_subsidiary_name

In [ ]:
main_df

,Name,Particulars of issued/paid-in capital,Proportion of equity interest held by the Group(%),Match,Matched subsidiary name
0,Tencent Computer,"RMB65,000,000",100%,No,
1,Tencent Technology,"USD2,000,000",100%,Yes,Tencent Technology (Beijing) Company Limited
2,Shenzhen Shiji Kaixuan Technology Company Limited,"RMB11,000,000",100%,No,
3,Tencent Cyber (Tianjin) Company Limited,"USD90,000,000",100%,Yes,Tencent Cyber (Shenzhen) Company Limited
4,Tencent Asset Management Limited,USD100,100%,No,
5,Tencent Technology (Beijing) Company Limited,"USD1,000,000",100%,Yes,Tencent Technology
6,Nanjing Wang Dian Technology Company Limited,"RMB10,290,000",100%,No,
7,Beijing BIZCOM Technology Company Limited,"RMB1,216,500,000",100%,No,
8,Beijing Starsinhand Technology Company Limited,"RMB10,000,000",100%,No,
9,Tencent Cyber (Shenzhen) Company Limited,"USD30,000,000",100%,Yes,Tencent Cyber (Tianjin) Company Limited


**4. Export dataframe as xlsx file**

In [ ]:
main_df.to_excel (r'/Users/eddytrang/Desktop/Tencent_sub0427.xlsx', index = False, header=True)